In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
import ast

# pip
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 8]

# DEV: override installed teslakit
import sys
sys.path.insert(0,'../../')

# teslakit 
from teslakit.project_site import Site
from teslakit.alr import ALR_WRP
from teslakit.custom_dateutils import xds_reindex_daily as xr_daily
from teslakit.custom_dateutils import xds_common_dates_daily as xcd_daily
from teslakit.custom_dateutils import xds2datetime as x2d
from teslakit.io.aux_nc import StoreBugXdset as sbxds


# --------------------------------------
# Site paths and parameters
data_folder = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
site = Site(data_folder, 'KWAJALEIN')

ST = site.pc.site                          # site database
PR = site.params                           # site parameters

# input files: historical MJO
p_mjo_hist = op.join(data_folder, 'database', 'MJO', 'MJO_hist.nc')

# input files
p_est_pred = ST.ESTELA.pred_slp
p_est_kma = op.join(p_est_pred, 'kma.nc')  # ESTELA + TCs Predictor
p_sst_PCA = ST.SST.pca                     # SST PCA

p_sst_PCs_sim_d = ST.SST.pcs_sim_d         # daily PCs sim
p_mjo_sim =  ST.MJO.sim                    # daily MJO sim

# output files
p_alr_covars =  ST.ESTELA.alrw             # alr wrapper
p_dwt_sim = ST.ESTELA.sim_dwt              # daily weather types simulated with ALR

# ALR parameters
alr_markov_order = int(PR.SIMULATION.alr_covars_markov)
alr_seasonality = ast.literal_eval(PR.SIMULATION.alr_covars_seasonality)
n_sim = int(PR.SIMULATION.n_sim)


In [2]:
# --------------------------------------
# Get data used to FIT ALR model and preprocess

# KMA: bmus (daily)
xds_KMA_fit = xr.open_dataset(p_est_kma)

# MJO: rmm1, rmm2 (daily)
xds_MJO_fit = xr.open_dataset(p_mjo_hist)
print(xds_MJO_fit)
print('')


# SST: PCs (annual)
xds_PCs = xr.open_dataset(p_sst_PCA)
sst_PCs = xds_PCs.PCs.values[:]
xds_PCs_fit = xr.Dataset(
    {
        'PC1': (('time',), sst_PCs[:,0]),
        'PC2': (('time',), sst_PCs[:,1]),
        'PC3': (('time',), sst_PCs[:,2]),
    },
    coords = {'time': xds_PCs.time.values[:]}
)

# reindex annual data to daily data
xds_PCs_fit = xr_daily(xds_PCs_fit)
print(xds_PCs_fit)


<xarray.Dataset>
Dimensions:  (time: 14343)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-04-08
Data variables:
    phase    (time) int64 ...
    rmm1     (time) float64 ...
    rmm2     (time) float64 ...
    mjo      (time) float64 ...

<xarray.Dataset>
Dimensions:  (time: 49674)
Coordinates:
  * time     (time) datetime64[ns] 1880-06-01 1880-06-02 ... 2016-06-01
Data variables:
    PC1      (time) float64 7.972 7.972 7.972 7.972 ... 55.53 55.53 55.53 -7.664
    PC2      (time) float64 1.668 1.668 1.668 1.668 ... 7.856 7.856 7.856 35.98
    PC3      (time) float64 -7.23 -7.23 -7.23 -7.23 ... 3.494 3.494 3.494 -9.33


In [3]:
# --------------------------------------
# Get data used to SIMULATE ALR model and preprocess

# MJO: rmm1, rmm2 (daily data)
xds_MJO_sim = xr.open_dataset(p_mjo_sim)
print(xds_MJO_sim)
print('')

# SST: PCs (daily data)
xds_PCs_sim = xr.open_dataset(p_sst_PCs_sim_d)
print(xds_PCs_sim)


<xarray.Dataset>
Dimensions:  (time: 365243)
Coordinates:
  * time     (time) object 2020-01-01 00:00:00 ... 3020-01-01 00:00:00
Data variables:
    mjo      (time) float32 ...
    phase    (time) float32 ...
    rmm1     (time) float32 ...
    rmm2     (time) float32 ...

<xarray.Dataset>
Dimensions:  (time: 365243)
Coordinates:
  * time     (time) object 2020-06-01 00:00:00 ... 3020-06-01 00:00:00
Data variables:
    PC1      (time) float32 ...
    PC2      (time) float32 ...
    PC3      (time) float32 ...


In [4]:
# --------------------------------------
# Mount covariates matrix

# available data:
# model fit: xds_KMA_fit, xds_MJO_fit, xds_PCs_fit
# model sim: xds_MJO_sim, xds_PCs_sim

# bmus fit
xds_BMUS_fit = xr.Dataset(
    {
        'bmus':(('time',), xds_KMA_fit['bmus'].values[:]),
    },
    coords = {'time': xds_KMA_fit.time.values[:]}
)

# covariates_fit
d_covars_fit = xcd_daily([xds_MJO_fit, xds_PCs_fit, xds_BMUS_fit])

# KMA dates
xds_BMUS_fit = xds_BMUS_fit.sel(time=slice(d_covars_fit[0],d_covars_fit[-1]))

# PCs covars 
cov_PCs = xds_PCs_fit.sel(time=slice(d_covars_fit[0],d_covars_fit[-1]))
cov_1 = cov_PCs.PC1.values.reshape(-1,1)
cov_2 = cov_PCs.PC2.values.reshape(-1,1)
cov_3 = cov_PCs.PC3.values.reshape(-1,1)

# MJO covars
cov_MJO = xds_MJO_fit.sel(time=slice(d_covars_fit[0],d_covars_fit[-1]))
cov_4 = cov_MJO.rmm1.values.reshape(-1,1)
cov_5 = cov_MJO.rmm2.values.reshape(-1,1)

# join covars 
# TODO: FER. NORMALIZO CONTRA TODO EL TIEMPO O EL TIEMPO DENTRO DE SIMULACION
cov_T = np.hstack((cov_1, cov_2, cov_3, cov_4, cov_5))

# normalize
cov_norm_fit = (cov_T - cov_T.mean(axis=0)) / cov_T.std(axis=0)
xds_cov_fit = xr.Dataset(
    {
        'cov_norm': (('time','n_covariates'), cov_norm_fit),
        'cov_names': (('n_covariates',), ['PC1','PC2','PC3','MJO1','MJO2']),
    },
    coords = {
        'time': d_covars_fit,
    }
)
print(xds_cov_fit)
print('')


# covariates: SIMULATION
d_covars_sim = xcd_daily([xds_MJO_sim, xds_PCs_sim])

# PCs covar 
cov_PCs = xds_PCs_sim.sel(time=slice(d_covars_sim[0],d_covars_sim[-1]))
cov_1 = cov_PCs.PC1.values.reshape(-1,1)
cov_2 = cov_PCs.PC2.values.reshape(-1,1)
cov_3 = cov_PCs.PC3.values.reshape(-1,1)

# MJO covars
cov_MJO = xds_MJO_sim.sel(time=slice(d_covars_sim[0],d_covars_sim[-1]))
cov_4 = cov_MJO.rmm1.values.reshape(-1,1)
cov_5 = cov_MJO.rmm2.values.reshape(-1,1)

# join covars (do not normalize simulation covariates)
cov_T_sim = np.hstack((cov_1, cov_2, cov_3, cov_4, cov_5))
xds_cov_sim = xr.Dataset(
    {
        'cov_values': (('time','n_covariates'), cov_T_sim),
    },
    coords = {
        'time': d_covars_sim,
    }
)
print(xds_cov_sim)
print('')


<xarray.Dataset>
Dimensions:    (n_covariates: 5, time: 11668)
Coordinates:
  * time       (time) datetime64[ns] 1979-02-12 1979-02-13 ... 2011-01-22
Dimensions without coordinates: n_covariates
Data variables:
    cov_norm   (time, n_covariates) float64 -0.1621 0.9171 ... -1.384 2.114
    cov_names  (n_covariates) <U4 'PC1' 'PC2' 'PC3' 'MJO1' 'MJO2'

<xarray.Dataset>
Dimensions:     (n_covariates: 5, time: 365091)
Coordinates:
  * time        (time) object 2020-06-01 2020-06-02 ... 3019-12-31 3020-01-01
Dimensions without coordinates: n_covariates
Data variables:
    cov_values  (time, n_covariates) float32 16.366308 7.2967243 ... -1.3139795



In [5]:
# --------------------------------------
# Autoregressive Logistic Regression

# available data:
# model fit: xds_KMA_fit, xds_cov_sim, num_clusters
# model sim: xds_cov_sim, sim_num, sim_years


# ALR terms
num_clusters = 42  # TODO NUM CLUSTERS AQUI EN .INI?
d_terms_settings = {
    'mk_order'  : alr_markov_order,
    'constant' : True,
    'long_term' : False,
    'seasonality': (True, alr_seasonality),
    'covariates': (True, xds_cov_fit),
}

# ALR wrapper
ALRW = ALR_WRP(p_alr_covars)
ALRW.SetFitData(num_clusters, xds_KMA_fit, d_terms_settings)

# ALR model fitting
fit_and_save = True 
if fit_and_save:
    ALRW.FitModel(max_iter=20000)
else:
    ALRW.LoadModel()

    
# Plot model p-values and params
#ALRW.Report_Fit()



Fitting autoregressive logistic model ...
Optimization done in 39.56 seconds

ALR model saved at /Users/nico/Projects/TESLA-kit/TeslaKit/data/sites/KWAJALEIN/ESTELA/alr_w/model.sav


In [6]:
# --------------------------------------
# Autoregressive Logistic Regression - simulate 

# launch simulation
dates_sim = d_covars_sim  # simulation dates
xds_alr = ALRW.Simulate(n_sim, dates_sim, xds_cov_sim, progress_bar=False)

# Store Daily Weather Types
xds_DWT_sim = xds_alr.evbmus_sims.to_dataset()
print(xds_DWT_sim)

# xarray.Dataset.to_netcdf() wont work with this time array and time dtype
sbxds(xds_DWT_sim, p_dwt_sim)
print('\nDWTs Simulation stored at:\n{0}'.format(p_dwt_sim))


ALR model fit   : 1979-02-12T00:00:00.000000000 --- 2011-01-22T00:00:00.000000000
ALR model sim   : 2020-06-01 00:00:00 --- 3020-01-01 00:00:00

Launching simulations...

<xarray.Dataset>
Dimensions:      (n_sim: 1, time: 365091)
Coordinates:
  * time         (time) datetime64[ns] 2020-06-01 ... 1850-11-22T00:50:52.580896768
Dimensions without coordinates: n_sim
Data variables:
    evbmus_sims  (time, n_sim) float64 9.0 22.0 27.0 28.0 ... 10.0 3.0 10.0 15.0

DWTs Simulation stored at:
/Users/nico/Projects/TESLA-kit/TeslaKit/data/sites/KWAJALEIN/ESTELA/DWT_sim.nc
